# **`02 Entrenar modelo`**

Con el directorio especificado se procede a entrenar

In [1]:
import numpy as np
import os
import cv2
import json
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical

In [2]:
import tensorflow as tf
from tensorflow.keras import layers

In [3]:
# Configuración
IMG_SIZE = 64
BATCH_SIZE = 32
EPOCHS = 20

In [4]:
# Función para cargar imágenes y etiquetas
def load_data(dir_path):
    images = []
    labels = []
    for person_name in os.listdir(dir_path):
        person_path = os.path.join(dir_path, person_name)
        if os.path.isdir(person_path):
            for img_name in os.listdir(person_path):
                img_path = os.path.join(person_path, img_name)
                img = cv2.imread(img_path)
                img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
                img = img[:, :, ::-1] / 255.0  # BGR to RGB and normalize
                images.append(img)
                labels.append(person_name)
    return np.array(images), np.array(labels)

In [5]:
# Definir el modelo
def create_model(num_classes):
    model = Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

In [6]:
def save_labels(labels, filename):
  # Convert NumPy integers to Python integers
  labels = {k: int(v) for k, v in labels.items()}

  with open(filename, 'w') as f:
      json.dump(labels, f)

In [7]:
from google.colab import drive
# Montar Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
# Cargar datos
# dir_path = '/content/drive/MyDrive/Personales/Estudio/BigData/Mod11/TareaFinal/personas_224x224'
dir_path = '/content/drive/MyDrive/ProyectoFotos/Celebrity_Faces_Dataset/personas_nuevas_224x224/'
images, labels = load_data(dir_path)

In [10]:
# Codificar etiquetas
le = LabelEncoder()
labels_encoded = le.fit_transform(labels)

# Guardar el mapeo de etiquetas
label_mapping = dict(zip(le.classes_, le.transform(le.classes_)))

In [11]:
label_mapping

{'Angelina Jolie': 0,
 'Brad Pitt': 1,
 'Denzel Washington': 2,
 'Hugh Jackman': 3,
 'Jennifer Lawrence': 4,
 'Johnny Depp': 5,
 'Kate Winslet': 6,
 'Leonardo DiCaprio': 7,
 'Megan Fox': 8,
 'Natalie Portman': 9,
 'Nicole Kidman': 10,
 'Robert Downey Jr': 11,
 'Sandra Bullock': 12,
 'Scarlett Johansson': 13,
 'Tom Cruise': 14,
 'Tom Hanks': 15,
 'Will Smith': 16}

In [12]:
# save_labels(label_mapping, '/content/drive/MyDrive/Personales/Estudio/BigData/Mod11/TareaFinal/face_recognition_model.json')
save_labels(label_mapping, '/content/drive/MyDrive/ProyectoFotos/JSON/face_recognition_model.json')

# Convertir a one-hot encoding
labels_one_hot = to_categorical(labels_encoded)

# Dividir datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(images, labels_one_hot, test_size=0.2, random_state=42)

In [13]:
# Crear y compilar el modelo
num_classes = len(np.unique(labels_encoded))
model = create_model(num_classes)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=EPOCHS, validation_split=0.2, batch_size=BATCH_SIZE)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 12s 216ms/step - accuracy: 0.0800 - loss: 2.8368 - val_accuracy: 0.1565 - val_loss: 2.7458
Epoch 2/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 12s 286ms/step - accuracy: 0.1396 - loss: 2.6565 - val_accuracy: 0.1794 - val_loss: 2.4508
Epoch 3/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 12s 339ms/step - accuracy: 0.2875 - loss: 2.2584 - val_accuracy: 0.3931 - val_loss: 2.1184
Epoch 4/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 19s 280ms/step - accuracy: 0.4659 - loss: 1.6888 - val_accuracy: 0.4427 - val_loss: 1.8842
Epoch 5/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 7s 205ms/step - accuracy: 0.6033 - loss: 1.3241 - val_accuracy: 0.5153 - val_loss: 1.6158
Epoch 6/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 11s 223ms/step - accuracy: 0.7052 - loss: 0.9721 - val_accuracy: 0.5687 - val_loss: 1.5727
Epoch 7/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 8s 242ms/step - accuracy: 0.7544 - loss: 0.7184 - val_accuracy: 0.6450 - val_loss: 1.4110
Epoch 8/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 9s 203ms/step - accuracy: 0.8651 - loss: 0.4332 - val_accurac

In [14]:
# Evaluar el modelo
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f'\nPrecisión en el conjunto de prueba: {test_acc}')

11/11 - 1s - 86ms/step - accuracy: 0.6911 - loss: 1.8868

Precisión en el conjunto de prueba: 0.6911314725875854


In [15]:
# Guardar el modelo
# model.save('/content/drive/MyDrive/Personales/Estudio/BigData/Mod11/TareaFinal/face_recognition_model.h5')
model.save('/content/drive/MyDrive/ProyectoFotos/Modelo/face_recognition_model.h5')
print("Modelo y etiquetas guardados exitosamente.")

Modelo y etiquetas guardados exitosamente.
